In [45]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import pickle

In [37]:
# get dataset
positivity = pd.read_csv("twitter_training.csv")
labels = ["ID", "entity", "sentiment", "text"]
positivity.columns.values[0] = "ID"
positivity.columns.values[1] = "entity"
positivity.columns.values[2] = "sentiment"
positivity.columns.values[3] = "text"
positivity = positivity.drop_duplicates()
positivity

,ID,entity,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [27]:
positivity["sentiment"].value_counts()

Negative      21787
Positive      19810
Neutral       17800
Irrelevant    12584
Name: sentiment, dtype: int64

In [28]:
positivity.drop(positivity[positivity['sentiment'] == 'Irrelevant'].index).reset_index()

,index,ID,entity,sentiment,text
0,0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...,...
59392,74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
59393,74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
59394,74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
59395,74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [29]:
X = positivity["text"]
y = positivity["sentiment"]
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(y)
y

array([3, 3, 3, ..., 3, 3, 3])

In [40]:
messages = []
for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', str(text))
    text = text.lower()
    messages.append(text)
messages

['i am coming to the borders and i will kill you all ',
 'im getting on borderlands and i will kill you all ',
 'im coming on borderlands and i will murder you all ',
 'im getting on borderlands   and i will murder you me all ',
 'im getting into borderlands and i can murder you all ',
 "so i spent a few hours making something for fun. . . if you don't know i am a huge  borderlands fan and maya is one of my favorite characters. so i decided to make myself a wallpaper for my pc. . here is the original image versus the creation i made    enjoy  pic.twitter.com/mlsi wf jg",
 "so i spent a couple of hours doing something for fun... if you don't know that i'm a huge   borderlands fan and maya is one of my favorite characters  i decided to make a wallpaper for my pc.. here's the original picture compared to the creation i made   have fun  pic.twitter.com / mlsi wf jg",
 "so i spent a few hours doing something for fun... if you don't know i'm a huge   borderlands fan and maya is one of my fav

In [41]:
np.array(messages).shape

(71981,)

In [48]:
cv = CountVectorizer()
X = cv.fit_transform(messages).toarray()
X.shape

MemoryError: Unable to allocate 6.12 GiB for an array with shape (40000, 20542) and data type int64